In [1]:
import pandas as pd

ds = pd.read_parquet('./results/ds.parquet')
elapse_time = pd.read_csv('./results/elapse_time.csv')

In [2]:
from bertopic import BERTopic
from gensim.models.ldamulticore import LdaMulticore
from utils import e_variant

def load_ldas():
    r = {} # result
    for v in ds:
        r[v] = LdaMulticore.load(f'./results/models_lda/{v}')
    return r

def load_bertopics():
    path_prefix = './results/models_bertopic/'
    r = {}
    for v in ds:
        r[v] = {}
        r[v] = BERTopic.load(f'{path_prefix}{v}')
    return r

In [3]:
ldas = load_ldas()
bertopics = load_bertopics()

In [4]:
from time import time
from gensim.corpora.dictionary import Dictionary
from utils import get_topics_lda, get_topics_bertopic, get_coherence, get_diversity

def evaluate_models(models, lda=False):
    r = []
    r_time = []
    for v in ds:
        t_start = time()
        docs = ds[v].dropna()
        if lda:
            topics = get_topics_lda(models[v], models[v].id2word)
            texts = docs
            dictionary = models[v].id2word
        else:
            topics = get_topics_bertopic(bertopics[v])
            analyzer = models[v].vectorizer_model.build_analyzer()
            texts = [' '.join(doc) for doc in docs]
            texts = [analyzer(doc) for doc in texts]
            dictionary = Dictionary(texts)
        c = get_coherence(
            topics=topics,
            texts=texts,
            dictionary=dictionary,
        )
        d = get_diversity(topics)
        t_end = time()
        r_time.append(t_end - t_start)
        r.append({
            'variant': v,
            'coherence': c,
            'diversity': d,
            'score': c * d
        })
    algorithm = 'lda' if lda else 'bertopic'
    elapse_time[f'{algorithm}_evaluation'] = pd.Series(r_time)
    return pd.DataFrame(r)

In [ ]:
import pandas as pd

evaluation_lda = pd.DataFrame(evaluate_models(ldas, lda=True))

In [6]:
evaluation_bertopic = pd.DataFrame(evaluate_models(bertopics))

In [7]:
elapse_time

,variant,nlp,preprocessing,lda_training,bertopic_training,lda_evaluation,bertopic_evaluation
0,dLWN,33.653473,0.081993,33.434384,53.977166,5.977065,4.743266
1,DLWN,33.653473,0.159020,33.695258,13.361671,5.901213,4.719315


In [8]:
evaluation_lda

,variant,coherence,diversity,score
0,dLWN,NaN,0.535714,NaN
1,DLWN,0.468277,0.535714,0.250863


In [9]:
evaluation_bertopic

,variant,coherence,diversity,score
0,dLWN,0.297210,1.0,0.297210
1,DLWN,0.254793,1.0,0.254793


In [10]:
elapse_time.to_csv('./results/elapse_time.csv', index=False)
evaluation_lda.to_csv('./results/evaluation_lda.csv', index=False)
evaluation_bertopic.to_csv('./results/evaluation_bertopic.csv', index=False)